In [ ]:
import requests
from bs4 import BeautifulSoup
import praw
import json
import datetime

# Scrape news headlines from Moneycontrol (example source for NSE-related news)
def scrape_moneycontrol_news():
    url = "https://www.moneycontrol.com/news/business/markets/"
    headers = {"User-Agent": "Mozilla/5.0"}
    
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("Failed to retrieve data")
        return []
    
    soup = BeautifulSoup(response.text, "html.parser")
    headlines = []
    
    for article in soup.find_all("li", class_="clearfix" ):
        title = article.find("h2")
        link = article.find("a")
        if title and link:
            headlines.append({"title": title.text.strip(), "url": link["href"]})
    
    return headlines

# Scrape Reddit posts from stock-related subreddits
def scrape_reddit():
    reddit = praw.Reddit(
        client_id="PVs4-U0vxQ56XsGDH6vZQg",
        client_secret="wTEdNfP1YRGlW4g-t7uXS5xhzb3y5A",
        user_agent="StockSentimentScraper by u/ApprehensiveDonut463"
    )
    
    subreddits = ["IndianStockMarket", "stocks", "StockMarketIndia"]
    posts = []
    
    for subreddit in subreddits:
        for submission in reddit.subreddit(subreddit).hot(limit=10):
            posts.append({
                "title": submission.title,
                "score": submission.score,
                "url": submission.url,
                "created_utc": datetime.datetime.fromtimestamp(submission.created_utc).isoformat()
            })
    
    return posts

if __name__ == "__main__":
    news_data = scrape_moneycontrol_news()
    reddit_data = scrape_reddit()
    
    with open("stock_sentiment_data.json", "w", encoding="utf-8") as f:
        json.dump({"news": news_data, "reddit": reddit_data}, f, ensure_ascii=False, indent=4)
    
    print("Data collected and saved successfully!")


: 